# Get data and packages

In [4]:
%%capture
%%bash
pip install -U pyarrow numpy koalas
# aws s3 cp  s3://xdss-public-datasets/demos/taxi_parquet/data_0.parquet datasets/taxi_parquet/

In [5]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os
import os


instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'spark-koalas'
data_path = 'datasets/taxi_parquet/'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 

benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks=benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]
          
def add_nan(name):
    for i in range(2):
        benchmarks['duration'].append(np.nan)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]

          
!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [6]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession

# Load data
data = ks.read_parquet(data_path)
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057928 with 18 columns


In [7]:
def read_file_parquet(df=None):
    return ks.read_parquet(data_path)

benchmark(read_file_parquet, df=data, name='read_file')

read_file took: 0.14801907539367676


0.14801907539367676

In [8]:
def count(df=None):
    return len(df)

benchmark(count, df=data, name='count')

count took: 0.49175596237182617


0.49175596237182617

In [9]:
def mean(df):
    return df.fare_amount.mean()

benchmark(mean, df=data, name='mean')

mean took: 2.7161142826080322


2.7161142826080322

In [10]:
def standard_deviation(df):
    return df.fare_amount.std()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 3.6770732402801514


3.6770732402801514

In [11]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean()

benchmark(mean_of_sum, df=data, name='sum columns mean')


sum columns mean took: 3.7391417026519775


3.7391417026519775

In [12]:
# lazy evaulation - instant
def sum_columns(df):
    return df.fare_amount + df.trip_distance

benchmark(sum_columns, df=data, name='sum columns')

sum columns took: 0.003270864486694336


0.003270864486694336

In [13]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean()

benchmark(mean_of_product, df=data, name='product columns mean')

product columns mean took: 3.737787961959839


3.737787961959839

In [14]:
# lazy evaulation - instant
def product_columns(df):
    return df.fare_amount * df.trip_distance

benchmark(product_columns, df=data, name='product columns')

product columns took: 0.0006079673767089844


0.0006079673767089844

In [15]:
# lazy evaulation - instant
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    return np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation')

arithmetic operation took: 0.13380718231201172


0.13380718231201172

In [16]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 
    return ret.mean()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')

arithmetic operation mean took: 88.87361717224121


88.87361717224121

In [17]:
def value_counts(df):
    counts = df.fare_amount.value_counts()
    l = counts.tolist()
    del l
    return counts

benchmark(value_counts, df=data, name='value counts')

value counts took: 16.39143180847168


16.39143180847168

In [18]:
def groupby_statistics(df):
    gb = df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              })
    p = gb.to_pandas()
    del p
    return gb
benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 32.093740701675415


32.093740701675415

In [19]:
other = ks.DataFrame(groupby_statistics(data).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

In [20]:
def join_count(df, other):
    return len(df.join(other, on='passenger_count'))

benchmark(join_count, data, name='join count', other=other)

join count took: 337.72805166244507


337.72805166244507

In [21]:
# lazy - instant
def join_data(df, other):
    joined = df.join(other, on='passenger_count')
    return joined 

benchmark(join_data, data, name='join', other=other)

join took: 0.17075157165527344


0.17075157165527344

## Filtered data

In [22]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 11 MB


In [ ]:
expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
                  (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
                  (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
                  (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

def filter_data(df):
    filtered = df[expr_filter]
    p = filter_data(data).to_pandas()
    del p
    return filtered

benchmark(filter_data, data, name='filter data')

In [ ]:
filtered = ks.DataFrame(filter_data(data).to_pandas())

del data
print(f"cleaned {gc.collect()} mb")

In [ ]:
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
benchmark(sum_columns, df=filtered, name='filtered sum columns')
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
benchmark(product_columns, df=filtered, name='filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
benchmark(complicated_arithmetic_operation, filtered, name='filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = ks.DataFrame(groupby_statistics(data).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])
benchmark(join_data, filtered, name='filtered join', other=other)
benchmark(join_count, filtered, name='filtered join count', other=other)
print(name)
get_results(benchmarks)